# Strucutred Output


In [8]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate,PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()


True

In [9]:
system_message_prompt = SystemMessagePromptTemplate.from_template("You are a helpful assistant that Gives States based on Country Name")
human_message_prompt = HumanMessagePromptTemplate.from_template("Country Name {question}, Give output in the JSON format Key Value Pair format Country: StateList")

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
question = "India"
final_prompt = prompt.format_messages(question=question)
print(final_prompt)



[SystemMessage(content='You are a helpful assistant that Gives States based on Country Name'), HumanMessage(content='Country Name India, Give output in the JSON format Key Value Pair format Country: StateList')]


In [10]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", convert_system_message_to_human=True)
response = llm.invoke(final_prompt)
print(response)

content='```json\n{\n  "India": [\n    "Andhra Pradesh",\n    "Arunachal Pradesh",\n    "Assam",\n    "Bihar",\n    "Chhattisgarh",\n    "Goa",\n    "Gujarat",\n    "Haryana",\n    "Himachal Pradesh",\n    "Jharkhand",\n    "Karnataka",\n    "Kerala",\n    "Madhya Pradesh",\n    "Maharashtra",\n    "Manipur",\n    "Meghalaya",\n    "Mizoram",\n    "Nagaland",\n    "Odisha",\n    "Punjab",\n    "Rajasthan",\n    "Sikkim",\n    "Tamil Nadu",\n    "Telangana",\n    "Tripura",\n    "Uttar Pradesh",\n    "Uttarakhand",\n    "West Bengal"\n  ]\n}\n```'


## Pydantic

In [11]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class CountryState(BaseModel):
    country: str = Field(..., description="The name of the country")
    states: list[str] = Field(..., description="A list of states in the country")

parser = PydanticOutputParser(pydantic_object=CountryState)

system_message_prompt = SystemMessagePromptTemplate.from_template("You are a helpful assistant that Gives States based on Country Name")
human_message_prompt = HumanMessagePromptTemplate.from_template("Country Name {question} \n {format_instructions}")


prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
question = "India"

    
print("Format Instructions:", parser.get_format_instructions())
final_prompt = prompt.format_messages(question=question, format_instructions=parser.get_format_instructions())

llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", convert_system_message_to_human=True)

chain = llm | parser

response = chain.invoke(final_prompt)
print(response)

print(response.country)
print(response.states)

Format Instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"country": {"description": "The name of the country", "title": "Country", "type": "string"}, "states": {"description": "A list of states in the country", "items": {"type": "string"}, "title": "States", "type": "array"}}, "required": ["country", "states"]}
```
country='India' states=['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'N